<a href="https://colab.research.google.com/github/youse0ng/AIProgramming/blob/main/AIProgramming2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인터넷 신규 가입 여부 예측

In [ ]:
# 필요라이브러리 불러오기
import pandas as pd

data_url='http://www-stat.wharton.upenn.edu/~waterman/DataSets/uva.txt'
df=pd.read_table(data_url)
df

11개의 특성과 19583개의 행 데이터

## 가정하기

Sexual Preference 성적 선호도와 Marital Status (결혼 여부)는 인터넷 가입여부와의 상관관계는 없을 것이다.

그래서 두개의 칼럼을 삭제하고 진행하겠다

In [ ]:
df=df.drop(columns=['Sexual Preference','Marital Status'])
df=df.drop(columns=['who'])

In [ ]:
df

## 데이터 칼럼을 한국어로 바꾸기

In [ ]:
df_changed=df.rename(columns={'Newbie': '신규 가입',
              'Age':'나이',
              'Gender':'성별',
              'Household Income': '가계 소득',
              'Country':'주',
              'Education Attainment':'취득 학위',
              'Major Occupation':'주요 직종',
              'Years on Internet': '인터넷 사용햇수'})


In [ ]:
df_changed

## 인터넷 신규 가입 여부에 대한 데이터 이해

In [ ]:
df_changed['신규 가입'].value_counts()

# 신규가입한 인원은 4962명 신규가입 안한 인원은 14621명

In [ ]:
df_changed['나이'].value_counts() # 나중에 cut으로 구간 생성하자

6살부터 많게는 79살까지 다양한 연령층이 존재 cut으로 분류할것

In [ ]:
df_changed['성별'].value_counts()

# 남성이 설문조사에 많이 참여
# 남성 13454, 여성: 6129 명

In [ ]:
df_changed['가계 소득'].value_counts()

In [ ]:
df_changed['주'].value_counts()

In [ ]:
df_changed['취득 학위'].value_counts()

In [ ]:
df_changed['주요 직종'].value_counts()

In [ ]:
df_changed['인터넷 사용햇수'].value_counts()

In [ ]:
df_changed

In [ ]:
df_changed.corr()
# 나이와 신규가입에 관한 상관관계는 0.069로 미미하다.

In [ ]:
# 성별과 신규가입의 어떤 범주형 관계가 있는가..?
pd.crosstab(df_changed['성별'],df_changed['신규 가입'])

# 신규가입을 안한 남성의 인원이 10000명

어린이 5~9

청소년 10 ~ 20

성인 20 ~ 60

노인 60~

In [ ]:
df_changed['나이']=pd.cut(x=df_changed['나이'],bins=[5,10,20,60,df_changed['나이'].max()],labels=['어린이','청소년','성인','노인'])

In [ ]:
df_age=df_changed['나이'].value_counts()
df_age.index

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.pie(df_age,labels=df_age.index,autopct='%.1f%%')

In [ ]:
# 나이와 신규가입의 관계
# 나이 묶기 [어린이, 청소년, 성인, 노인]
# 성인의 신규가입이 가장많음 두번째는 청소년
pd.crosstab(df_changed['나이'],df_changed['신규 가입'])

In [ ]:
df_changed.info()

## 해당 데이터프레임의 결측치 존재확인

In [ ]:
df_changed.info()

나이, 가계소득에서 데이터 결측치가 있음을 확인할 수 있다.


In [ ]:
df_changed.isnull().sum()

나이 :561개의 결측치, 가계소득: 3185개의 결측치가 존재

데이터의 결측치를 제거하고 싶지만, 정보 손실이 심할거란 생각에 Mean 해당 칼럼의 mean을 구하여 결측치를 대신할 것이다.

In [ ]:
a=df[df_changed['나이'].isnull()==True].index
df_changed=df_changed.drop(index=a,axis=0)

In [ ]:
df_changed.isnull().sum()
# 나이의 결측치 제거

df_changed['가계 소득']에 있는 NaN에 최빈값으로 대체하기

In [ ]:
df_changed['가계 소득'].fillna(df_changed['가계 소득'].mode()[0],inplace=True)

In [ ]:
df_changed

In [ ]:
df=pd.get_dummies(df_changed,columns=['성별'])
df.head()

In [ ]:
# 사이킷런으로 레이블 인코딩하기
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
category_col=['나이','가계 소득','주','취득 학위','주요 직종','인터넷 사용햇수']
# labelEncoder로 나머지 칼럼들 레이블 인코딩하기
for col in category_col:
  df[col]=le.fit_transform(df[col])

df.head()

# 학습데이터와 정답데이터 분리

In [ ]:
# 학습 데이터
X=df.drop(columns=['신규 가입'])
X

In [ ]:
# 정답 데이터
y=df['신규 가입']
y

In [ ]:
# 학습데이터 정답데이터 분리 및 검증데이터 분리
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,
                                               test_size=0.2,
                                               random_state=42,
                                               shuffle=True)
X_train.shape, X_test.shape, y_train.shape ,y_test.shape

# 모델 선언